# **GPT-fuego (LLM Agent Challenge)** by Nathan Roll

## **Scrape Data Science Notebooks**



In [ ]:
from tqdm import tqdm
import os

FOLDER_PATH = "/content/drive/Shareddrives/Notebooks"

In [ ]:
# %pip install -q google-colab-selenium[undetected] # Required to bypass captcha on Kaggle
# import google_colab_selenium as gs

In [ ]:
# urls = ['https://www.kaggle.com/code/dczerniawko/fifa19-analysis',
#         'https://www.kaggle.com/code/ekami66/detailed-exploratory-data-analysis-with-python',
#         'https://www.kaggle.com/code/anokas/data-exploration-analysis',
#         'https://www.kaggle.com/code/aeryan/spotify-music-analysis',
#         'https://www.kaggle.com/code/anokas/data-analysis-xgboost-starter-0-35460-lb',
#         'https://www.kaggle.com/code/edhirif/predict-the-causes-of-wildfires-using-python',
#         'https://www.kaggle.com/code/umerkk12/credit-card-predictive-analysis',
#         'https://www.kaggle.com/code/dansbecker/wildfire-analysis',
#         'https://www.kaggle.com/code/sentdex/first-pass-through-data-w-3d-convnet',
#         'https://www.kaggle.com/code/allunia/protein-atlas-exploration-and-baseline',
#         'https://www.kaggle.com/code/helgejo/an-interactive-data-science-tutorial',
#         'https://www.kaggle.com/code/goyalshalini93/car-price-prediction-linear-regression-rfe',
#         'https://www.kaggle.com/code/caesarlupum/ashrae-start-here-a-gentle-introduction',
#         'https://www.kaggle.com/code/go1dfish/clear-mask-visualization-and-simple-eda',
#         'https://www.kaggle.com/code/thykhuely/mercari-interactive-eda-topic-modelling',
#         'https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial',
#         'https://www.kaggle.com/code/kashnitsky/topic-9-part-1-time-series-analysis-in-python',
#         'https://www.kaggle.com/code/residentmario/grouping-and-sorting',
#         'https://www.kaggle.com/code/ratthachat/aptos-eye-preprocessing-in-diabetic-retinopathy',
#         'https://www.kaggle.com/code/codename007/home-credit-complete-eda-feature-importance',
#         'https://www.kaggle.com/code/janiobachmann/lending-club-risk-analysis-and-metrics',
#         'https://www.kaggle.com/code/etakla/exploring-the-dataset-bivariate-analysis',
#         'https://www.kaggle.com/code/nancyalaswad90/analysis-breast-cancer-prediction-dataset',
#         'https://www.kaggle.com/code/tirsogomez/banking-dataset-analysis',
#         'https://www.kaggle.com/code/carmelgafa/ibm-hr-dataset-analysis-and-prediction',
#         'https://www.kaggle.com/code/jneupane12/analysis-of-movielens-dataset-beginner-sanalysis',
#         'https://www.kaggle.com/code/volodymyrgavrysh/bank-marketing-campaigns-dataset-analysis',
#         'https://www.kaggle.com/code/qusaybtoush1990/airbnb-analysis-dataset',
#         'https://www.kaggle.com/code/gsdeepakkumar/e-commerce-dataset-analysis',
#         'https://www.kaggle.com/code/lalitharajesh/iris-dataset-exploratory-data-analysis']

# for url in tqdm(urls):
#   driver = gs.Chrome()
#   driver.get(url)
#   html = driver.page_source
#   driver.quit()

#   url_stem = url.split("/")[-1]
#   filepath = os.path.join(FOLDER_PATH, url_stem + '.ipynb')

#   with open(filepath, "w") as f:
#     f.write(html)

## **Load and Parse Notebooks for Fine-tuning**

In [ ]:
import json
combined_examples = []

def format_for_OpenAI(input, code=True):
  if code:
    return f"""```python\n{''.join(input)}\n```"""
  else:
    return ''.join(input)

for nb in tqdm(os.listdir(FOLDER_PATH)):
  if nb.endswith('.ipynb'):
    with open(FOLDER_PATH+'/'+nb) as f:
      raw_text = f.read()

    cells = json.loads(raw_text)['cells']

    examples = []

    for i in range(len(cells) - 2):
      if cells[i]['cell_type'] == 'markdown' and cells[i+1]['cell_type'] == 'code' and cells[i+1]['cell_type'] == 'markdown':
        examples.append((cells[i]['source'], cells[i+1]['source']))
      if cells[i]['cell_type'] == 'markdown' and cells[i+1]['cell_type'] == 'code' and cells[i+1]['cell_type'] == 'code':
        examples.append((cells[i]['source'], cells[i+1]['source'] + cells[i+2]['source']))

    examples = [(format_for_OpenAI(examples[0][1], code=True), format_for_OpenAI(e[0]), format_for_OpenAI(e[1], code=True)) for  e in examples][1:] # add first code cell as context at 0 index for input into prompt
    combined_examples.extend(examples)

100%|██████████| 31/31 [00:01<00:00, 25.04it/s]


In [ ]:
import random

random.shuffle(combined_examples)

validation_set_size = int(len(combined_examples) * 0.15)
print(f'val length: {validation_set_size}')

validation_set = combined_examples[:validation_set_size]
training_set = combined_examples[validation_set_size:]

val length: 93


In [ ]:
for s in ['validation_set', 'training_set']:

  with open(f'{s}.jsonl', 'w') as file:
    v = eval(s)
    for i in range(len(v)):
      item = {
        "messages": [
          {"role": "system", "content": "You are a data science assistant. Please generate python code to answer the user's prompts."},
          {"role": "user", "content": f"Context: {v[i][0]}, Prompt: {v[i][1]}"},
          {"role": "assistant", "content": str(v[i][2])}
        ]
      }

      json_line = json.dumps(item)
      file.write(json_line + '\n')

## **Fine-tune GPT 3.5 Turbo**

In [ ]:
!pip install --upgrade openai langchain

import openai
from tqdm import tqdm

OPENAI_KEY = 'YOUR_KEY_HERE'

client = openai.OpenAI(api_key = OPENAI_KEY)

In [ ]:
# train_dev = client.files.create(
#   file=open("training_set.jsonl", "rb"),
#   purpose="fine-tune"
# )

# valid = client.files.create(
#   file=open("validation_set.jsonl", "rb"),
#   purpose="fine-tune"
# )

# client.fine_tuning.jobs.create(
#   training_file=train_dev.id,
#   validation_file=valid.id,
#   suffix="gpt_fuego_v1",
#   model="gpt-3.5-turbo-0125",
#   hyperparameters={
#     "n_epochs":2
#   }
# )

## **Use fine-tuned model**

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Change the path to point to the correct location of the sqlite database file.
db_path = '/content/drive/Shareddrives/Aira- Wildfire/FPA_FOD_20170508.sqlite'

# Connect to the sqlite database file and retrieve data as a Pandas dataframe.
cnx = sqlite3.connect(db_path)
sql = "SELECT * from Fires"
df = pd.read_sql_query(sql, cnx)

In [ ]:
df.drop(columns=['Shape'], inplace=True)

In [ ]:
def gen_sql(prompt):
  response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    temperature = 0,
    max_tokens = 50,
    messages=[
      {"role": "system", "content": "You are a column selecting agent. You respond only with a tab-separated list of column names that could help answer the prompt and nothing more. Include too many column names instead of too few."},
      {"role": "user", "content": f"Prompt: {prompt}, Column Options: OBJECTID, FOD_ID, FPA_ID, SOURCE_SYSTEM_TYPE, SOURCE_SYSTEM,NWCG_REPORTING_AGENCY, NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME, SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME, LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID, FIRE_CODE, FIRE_NAME,ICS_209_INCIDENT_NUMBER, ICS_209_NAME, MTBS_ID, MTBS_FIRE_NAME,COMPLEX_NAME, FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY,DISCOVERY_TIME, STAT_CAUSE_CODE, STAT_CAUSE_DESCR, CONT_DATE,CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE,LONGITUDE, OWNER_CODE, OWNER_DESCR, STATE, COUNTY,FIPS_CODE, FIPS_NAME"}
    ]
  )

  raw = response.choices[0].message.content
  sql = 'SELECT ' + ', '.join(raw.split('\t')) + ' FROM `translate-413521.wildfire_aira.Fire`'
  return sql

In [ ]:
def ret_v(context, prompt):
  response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:nathannet:gpt-fuego-v1:9BoT37Uh",
    temperature = 0.15,
    max_tokens = 500,
    messages=[
      {"role": "system", "content": "You are a data science assistant. Please generate python code to answer the user's prompts."},
      {"role": "user", "content": f"Context: {context}, Prompt: {prompt} Create a result with only a single line of code using 'df' as an input."}
    ]
  )

  return response.choices[0].message.content

In [ ]:
def value_pipeline(code):
  if '```python' in code:
    out = eval(code.split('```python\n')[1].split('#')[0].split('```')[0].split('\n')[0])
  else:
    out = eval(code)
  return out

In [ ]:
def fix_secure(code):
  response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  temperature = 0.1,
  max_tokens = 500,
  messages=[
    {"role": "system", "content": "You are a code fixing assistant. Respond only in correct python syntax. If the code is already correct, return the exact input."},
    {"role": "user", "content": code}
  ]
  )

  return response.choices[0].message.content

In [ ]:
import concurrent.futures

def execute_code_in_pipeline(context):
  code = ret_v(context, prompt + ' Do not return a plot.')
  code_fixed = fix_secure(code)
  # print(code)
  outs = {}
  for pipe in ['value_pipeline']:
    try:
      # Dynamically call the pipeline functions with eval
      outs[pipe] = eval(f'{pipe}(code_fixed)')
    except Exception as e:
      outs[pipe] = f'error: {str(e)}'
  return outs

def multithread_code_output(context):
  with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(execute_code_in_pipeline, context) for _ in range(2)]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

  return results

In [ ]:
def package_output(prompt, results):
  response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    temperature = 0,
    max_tokens = 1000,
    messages=[
      {"role": "system", "content": "You are a data analysis assistant. Please use the outputs to create a succinct, correct answer to the question using only the info you need."},
      {"role": "user", "content": f'Question: {prompt}, Outputs: {results}.'}
    ]
  )

  return response.choices[0].message.content

## **Connect to Google Bigquery Wildfire DS**

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

from google.cloud import bigquery

# Initialize a BigQuery client
client_bq = bigquery.Client()

In [ ]:
prompt = 'What is the average size of a wildfire in California versus Texas?'
query = gen_sql(prompt)
query_job = client_bq.query(query)
df = query_job.to_dataframe()

wildfire_context = f'''```python\n# Connect to the sqlite db file and retrieve data as Pandas data frame.
cnx = sqlite3.connect('archive/FPA_FOD_20170508.sqlite')
sql = "select * from fires"
df = pd.read_sql_query(sql, cnx)\n# Cols = {df.columns}```'''
results = multithread_code_output(wildfire_context)

o = package_output(prompt, results)

## **Connect to APIs**

In [ ]:
!pip install census
!pip install us

In [ ]:
import requests, json
from census import Census
from us import states

CENSUS_API_KEY = 'YOUR_KEY_HERE'
OPEN_WEATHER_API_KEY = 'YOUR_KEY_HERE'


weather_base_url = "http://api.openweathermap.org/data/2.5/weather?"
weather_api_key = OPEN_WEATHER_API_KEY

value_keys = {
    "B06011_001E": "Median Income",
    "B18120_002E": "Size of labor force",
    "B18120_012E": "Unemployed"
}

def get_economy(state_code, year=None):
  if year:
    c = Census(CENSUS_API_KEY, year=year) # need to re-init each time to avoid sync errors :(
  else:
    c = Census(CENSUS_API_KEY)

  r = c.acs1.get(('NAME', 'B06011_001E', 'B18120_002E', 'B18120_012E'), #combed through ACS tables for fields: https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html
            {'for': f"state:{eval(f'states.{state_code}.fips')}"})[0]

  r = {k if k not in value_keys else value_keys[k]: v for k, v in r.items()}
  r['Unemployment rate'] = round(r['Unemployed'] / r['Size of labor force'], 4)
  return r


def kelvin_to_fahrenheit(K):
    F = (K * (9/5)) - 459.67
    return round(F, 4)

def get_weather(place, weather_api_key):

  complete_url = weather_base_url + "appid=" + weather_api_key + "&q=" + place

  response = requests.get(complete_url)
  x = response.json()

  d = {k: kelvin_to_fahrenheit(v) if 200 < v < 400 else v for k, v in x['main'].items()}
  d['description_of_weather'] = x['weather'][0]['description']

  return d

In [ ]:
def API_router(prompt):
  response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    temperature = 0,
    max_tokens = 1000,
    messages=[
      {"role": "system", "content": "You are an API assistant which receives a prompt and determines if it involves either a weather API or a census API. The census API only returns information on 'median_income' and 'unemployment_rate' and the weather API only returns the following fields: 'temp' 'feels_like' 'temp_min' 'temp_max' 'pressure' 'humidity' 'description_of_weather'. Respond with either 'census','weather','both', or 'niether' given the prompt. Also include the year, state, and state code if applicable. Do not include any other text. Examples: Prompt: 'Which state has the most whales and how hot was it in 2019?' Response: 'weather 2019 NA NA', Prompt: 'How many ducks are in CA and what was their unemployment rate?' Response: 'census NA California CA'"},
      {"role": "user", "content": f'Prompt: {prompt}'}
    ]
  )

  return response.choices[0].message.content

In [ ]:
def get_api(prompt):
  try:
    routed = API_router(prompt)
    print(routed)
    api, year, state, state_code = routed.split(' ')
    if year == 'NA':
      year = None
    else:
      year = int(year)

    if api == 'niether':
      pass
    elif api == 'census':
      r = get_economy(state_code, year)
    elif api == 'weather':
      r = get_weather(state, weather_api_key)
    elif api == 'both':
      r = {**get_economy(state, year), **get_weather(state, weather_api_key)}
    return r
  except ValueError:
    return None

In [ ]:
get_api('How many ducks were in CA in 2005 and what was their max temperature?')

weather 2005 California CA


{'temp': 75.416,
 'feels_like': 75.002,
 'temp_min': 72.662,
 'temp_max': 80.708,
 'pressure': 1014,
 'humidity': 50,
 'description_of_weather': 'clear sky'}

In [ ]:
get_weather('California', weather_api_key)

{'temp': 76.442,
 'feels_like': 76.136,
 'temp_min': 69.656,
 'temp_max': 79.826,
 'pressure': 1014,
 'humidity': 50,
 'desc': 'clear sky'}

In [ ]:
get_economy('CA', year=2018)

{'NAME': 'California',
 'Median Income': 32155.0,
 'Size of labor force': 18840089.0,
 'Unemployed': 1035440.0,
 'state': '06',
 'Unemployment rate': 0.055}

In [ ]:
get_economy('CA', year=2009)

{'NAME': 'California',
 'Median Income': 26130.0,
 'Size of labor force': 17718670.0,
 'Unemployed': 1966111.0,
 'state': '06',
 'Unemployment rate': 0.111}

## **Prompt ideas**

In [ ]:
# What was the mean size of a wildfire in California versus Texas in 1999?
# Which 6 states had the most wildfires and how many fires did they have?
# What was the most common name for a fire in 2008?
# What time of day is the most common to discover a fire?
# What was the average number of fires per state in 1999?
# How many fires between 2002 and 2004 were between 30 and 40 degrees latitude?

# Which state had the most wildfires in 2009? What was their median income and max temperature that year?